In [1]:
# Start by importing necessary libraries
import numpy as np
import time 
from tqdm import tqdm

import torch 
import torch.nn as nn
import torch.nn.functional as F

# Check for cuda
device = 'cuda' if torch.cuda.is_available() else 'cpu'
print(device)

# Setting important hyperparameters
batch_size = 16 # how many independent sequences will we process in parallel?
block_size = 32 # what is the maximum context length for predictions?
max_iters = 10000
eval_interval = 100
learning_rate = 1e-4
eval_iters = 200
n_embd = 64
n_heads = 4
dropout = 0.2

cuda


### Step 1: Looking at the text file

In [2]:
with open('/Net/Groups/BGI/scratch/ppandey/LLMs_Playground/The_Great_Gatsby.txt', 'r', encoding='utf-8') as file:
    txt_file = file.read()
print(len(txt_file))

270153


### Step 2: Construct a vocabulary

In [3]:
# Get unique characters
chars = sorted(set(txt_file))
print(chars)
vocabulary_size = len(chars)
print('Total number of unique characters/Vocabulary size:', vocabulary_size)

['\n', ' ', '!', '$', '(', ')', '*', ',', '-', '.', '0', '1', '2', '3', '4', '5', '6', '7', '8', '9', ':', ';', '?', 'A', 'B', 'C', 'D', 'E', 'F', 'G', 'H', 'I', 'J', 'K', 'L', 'M', 'N', 'O', 'P', 'Q', 'R', 'S', 'T', 'U', 'V', 'W', 'X', 'Y', '[', ']', 'a', 'b', 'c', 'd', 'e', 'f', 'g', 'h', 'i', 'j', 'k', 'l', 'm', 'n', 'o', 'p', 'q', 'r', 's', 't', 'u', 'v', 'w', 'x', 'y', 'z', 'ç', 'é', 'ê', 'ô', '\u200a', '—', '‘', '’', '“', '”', '…', '\ufeff']
Total number of unique characters/Vocabulary size: 88


In [4]:
print(list(enumerate(chars)))

[(0, '\n'), (1, ' '), (2, '!'), (3, '$'), (4, '('), (5, ')'), (6, '*'), (7, ','), (8, '-'), (9, '.'), (10, '0'), (11, '1'), (12, '2'), (13, '3'), (14, '4'), (15, '5'), (16, '6'), (17, '7'), (18, '8'), (19, '9'), (20, ':'), (21, ';'), (22, '?'), (23, 'A'), (24, 'B'), (25, 'C'), (26, 'D'), (27, 'E'), (28, 'F'), (29, 'G'), (30, 'H'), (31, 'I'), (32, 'J'), (33, 'K'), (34, 'L'), (35, 'M'), (36, 'N'), (37, 'O'), (38, 'P'), (39, 'Q'), (40, 'R'), (41, 'S'), (42, 'T'), (43, 'U'), (44, 'V'), (45, 'W'), (46, 'X'), (47, 'Y'), (48, '['), (49, ']'), (50, 'a'), (51, 'b'), (52, 'c'), (53, 'd'), (54, 'e'), (55, 'f'), (56, 'g'), (57, 'h'), (58, 'i'), (59, 'j'), (60, 'k'), (61, 'l'), (62, 'm'), (63, 'n'), (64, 'o'), (65, 'p'), (66, 'q'), (67, 'r'), (68, 's'), (69, 't'), (70, 'u'), (71, 'v'), (72, 'w'), (73, 'x'), (74, 'y'), (75, 'z'), (76, 'ç'), (77, 'é'), (78, 'ê'), (79, 'ô'), (80, '\u200a'), (81, '—'), (82, '‘'), (83, '’'), (84, '“'), (85, '”'), (86, '…'), (87, '\ufeff')]


### Step 3: Characterwise tokensiation

In [5]:
# Create encoder and decoder for characterwise tokenisation
string_to_int = {
    ch:i for i,ch in enumerate(chars)
}

int_to_string = {
    i:ch for i,ch in enumerate(chars)
}

charwise_encoder = lambda input_word: [string_to_int[char] for char in input_word]
charwise_decoder = lambda input_tokenised_word: ''.join([int_to_string[i] for i in input_tokenised_word])

In [6]:
# Experimenting with tokeniser
encoded_Jagjit_Singh = charwise_encoder('Jagjit Singh')
print(encoded_Jagjit_Singh)

[32, 50, 56, 59, 58, 69, 1, 41, 58, 63, 56, 57]


In [7]:
decoded_Jagjit_Singh = charwise_decoder(encoded_Jagjit_Singh)
print(decoded_Jagjit_Singh)

Jagjit Singh


In [8]:
# Convert the txt_file from array/list to tensor
data = torch.tensor(charwise_encoder(txt_file), dtype=torch.long)
print(data[:50])

tensor([87, 31, 63,  1, 62, 74,  1, 74, 64, 70, 63, 56, 54, 67,  1, 50, 63, 53,
         1, 62, 64, 67, 54,  1, 71, 70, 61, 63, 54, 67, 50, 51, 61, 54,  1, 74,
        54, 50, 67, 68,  1, 62, 74,  1, 55, 50, 69, 57, 54, 67])


### Step 4: Splitting the dataset 

In [9]:
# Split the dataset
train_percentage_split = int(0.9*len(data))
train_data = data[:train_percentage_split] # 90% training 
val_data = data[train_percentage_split:]

In [10]:
x = train_data[:block_size]
y = train_data[1:block_size+1]

for i in range(block_size):
    context = x[:i+1]
    targ = y[i]
    print('context:', context, ';', 'target:', targ)

context: tensor([87]) ; target: tensor(31)
context: tensor([87, 31]) ; target: tensor(63)
context: tensor([87, 31, 63]) ; target: tensor(1)
context: tensor([87, 31, 63,  1]) ; target: tensor(62)
context: tensor([87, 31, 63,  1, 62]) ; target: tensor(74)
context: tensor([87, 31, 63,  1, 62, 74]) ; target: tensor(1)
context: tensor([87, 31, 63,  1, 62, 74,  1]) ; target: tensor(74)
context: tensor([87, 31, 63,  1, 62, 74,  1, 74]) ; target: tensor(64)
context: tensor([87, 31, 63,  1, 62, 74,  1, 74, 64]) ; target: tensor(70)
context: tensor([87, 31, 63,  1, 62, 74,  1, 74, 64, 70]) ; target: tensor(63)
context: tensor([87, 31, 63,  1, 62, 74,  1, 74, 64, 70, 63]) ; target: tensor(56)
context: tensor([87, 31, 63,  1, 62, 74,  1, 74, 64, 70, 63, 56]) ; target: tensor(54)
context: tensor([87, 31, 63,  1, 62, 74,  1, 74, 64, 70, 63, 56, 54]) ; target: tensor(67)
context: tensor([87, 31, 63,  1, 62, 74,  1, 74, 64, 70, 63, 56, 54, 67]) ; target: tensor(1)
context: tensor([87, 31, 63,  1, 62, 

In [11]:
# Lets create `get_batch` function. Kind of dataloader
def get_batch(split):
    data = train_data if split=='train' else val_data
    random_indexes = torch.randint(0, (len(data) - block_size), (batch_size,))
    # print(random_indexes)
    x = torch.stack([data[i:i+block_size] for i in random_indexes])
    y = torch.stack([data[i+1:i+block_size+1] for i in random_indexes])
    # move the data to gpus
    x, y = x.to(device), y.to(device)
    return x, y

In [12]:
# Working with get_batch function
x, y = get_batch('train')
print('context/inputs:')
print(x)
print('target:')
print(y)

# We will have 4 random int values which is the starting point of each batch and have corresponding target

context/inputs:
tensor([[62, 68, 54, 61, 55,  1, 58, 63, 69, 64,  1, 50,  1, 68, 57, 50, 53, 64,
         72,  1, 50, 63, 53,  7,  0, 72, 57, 58, 61, 54,  1, 26],
        [70, 67,  1, 64, 55,  1, 69, 57, 54,  1, 72, 50, 61, 61, 68,  9,  1, 23,
          0, 72, 57, 58, 69, 54,  1, 50, 68, 57, 54, 63,  1, 53],
        [74,  1, 57, 64, 70, 68, 54,  1, 61, 64, 64, 60, 68,  1, 72, 54, 61, 61,
          7,  1, 53, 64, 54, 68, 63, 83, 69,  1, 58, 69, 22, 85],
        [58, 63, 56,  1, 69, 64,  1, 69, 57, 54,  1, 40, 54, 53,  0, 25, 67, 64,
         68, 68,  1, 69, 64,  1, 62, 50, 60, 54,  1, 51, 50, 63],
        [53,  7,  1, 84, 27, 73, 52, 70, 68, 54,  0, 62, 54, 85, 81, 51, 70, 69,
          1, 69, 57, 58, 68,  1, 69, 58, 62, 54,  1, 63, 64,  1],
        [ 9,  1, 30, 54, 67,  1, 54, 74, 54, 68,  1, 55, 54, 61, 61,  1, 64, 63,
          1, 32, 64, 67, 53, 50, 63,  1, 50, 63, 53,  1, 62, 54],
        [57, 50, 69,  1, 52, 64, 67, 67, 54, 52, 69, 54, 53,  1, 51, 67, 64, 72,
          1, 64, 55, 

### Step 6: Creating the Bigram class

Just for the context on what does `nn.Embedding` give as an output:

```python
vocab_size = 4000
embedding_dim = 5 
# In a sense, this is 5 dimensional space where each element of vocab is represented by a vector in that space.

# Initialise the embedding layer
embedding = nn.Embedding(vocab_size, embedding_dim)

# Create some example input data
input_data = torch.LongTensor([[1, 2, 3, 4], [5, 6, 7, 8]])

# Pass the input data through the embedding layer
embedded_data = embedding(input_data)

print(embedded_data.shape)
# 2 is the batch size, 4 is the sequence length, and 5 is the embedding dimension
```

In [13]:
@torch.no_grad()
def estimate_loss():
    out = {}
    model.eval() # good practice: keep in mind the mode we are in. this helps us understand/build our model correctly. 
    for split in ['train', 'val']:
        losses = torch.zeros(eval_iters)
        for k in range(eval_iters):
            X, Y = get_batch(split)
            logits, loss = model(X, Y)
            losses[k] = loss.item()
        out[split] = losses.mean()
    model.train()
    return out

Basically, I want my model to informed about the past information. Taking the weighted sum of the past. (Not the most efficient way but it works)

In [14]:
# Maths behind self-attention: 
B, T, C = 4, 8, 32
random_tensor = torch.randn((B, T, C))

# Create a single head attention 
head_size = 16
key = nn.Linear(C, head_size, bias=False)
query = nn.Linear(C, head_size, bias=False)
value = nn.Linear(C, head_size, bias=False)

k = key(random_tensor) # (B, T, 16)
q = query(random_tensor) # (B, T, 16)
wei = q @ k.transpose(-2, -1) # (B, T, 16) * (B, 16, T) --> (B, T, T)

# Now, we have to mix the information i.e past and present which will fed into our model.
# As discussed, to combine the informations we use weighted sum approach. Clever use of matrix multiplication and softmax for normalisation
tril = torch.tril(torch.ones(T, T))
# wei = torch.zeros((T, T)) # generally, this shouldnt be uniform. therefore, implemented key-query approach
wei = wei.masked_fill(tril==0, float('-inf'))
wei = F.softmax(wei, dim=-1) # sum of each row will be equal to 1

v = value(random_tensor)
out = wei @ v # This is something which is aggregated
x_bag_of_words = wei @ random_tensor # x or random_tensor is something which private in context of a particular token
# print(x_bag_of_words.shape)
print(wei[0]) # looking at single batch 
# Since we will have different tokens in different position for each batch. therefore, we will have different weight distribution

tensor([[1.0000e+00, 0.0000e+00, 0.0000e+00, 0.0000e+00, 0.0000e+00, 0.0000e+00,
         0.0000e+00, 0.0000e+00],
        [5.4649e-01, 4.5351e-01, 0.0000e+00, 0.0000e+00, 0.0000e+00, 0.0000e+00,
         0.0000e+00, 0.0000e+00],
        [5.6845e-01, 6.0980e-02, 3.7056e-01, 0.0000e+00, 0.0000e+00, 0.0000e+00,
         0.0000e+00, 0.0000e+00],
        [1.6899e-02, 1.4698e-01, 4.3027e-01, 4.0585e-01, 0.0000e+00, 0.0000e+00,
         0.0000e+00, 0.0000e+00],
        [2.7021e-01, 6.0117e-02, 3.2039e-01, 4.4503e-02, 3.0478e-01, 0.0000e+00,
         0.0000e+00, 0.0000e+00],
        [2.2065e-02, 7.7512e-01, 2.7749e-02, 8.8687e-04, 1.3792e-01, 3.6263e-02,
         0.0000e+00, 0.0000e+00],
        [1.4623e-02, 3.7534e-01, 3.0217e-02, 7.4997e-03, 2.9236e-02, 2.6814e-01,
         2.7495e-01, 0.0000e+00],
        [1.0809e-01, 1.9375e-01, 6.8777e-02, 2.8949e-02, 2.2153e-02, 2.7387e-01,
         2.4071e-01, 6.3709e-02]], grad_fn=<SelectBackward0>)


Main idea behind Self-Attention: Gather information from the past in a data dependent way!

How does it solve this?
Every single token at each position emits two vectors:
- Query: What am I looking for?
- Key: What do I contain!

We do the dot product of query with each of the key which ultimately is my `wei`.


Notes:
- Attention is a **communication mechanism**. Can be seen as nodes in a directed graph looking at each other and aggregating information with a weighted sum from all nodes that point to them, with data-dependent weights.
- There is no notion of space. Attention simply acts over a set of vectors. This is why we need to positionally encode tokens.
- Each example across batch dimension is of course processed completely independently and never "talk" to each other
- In an "encoder" attention block just delete the single line that does masking with `tril`, allowing all tokens to communicate. This block here is called a "decoder" attention block because it has triangular masking, and is usually used in autoregressive settings, like language modeling.
- "self-attention" just means that the keys and values are produced from the same source as queries. In "cross-attention", the queries still get produced from x, but the keys and values come from some other, external source (e.g. an encoder module)
- "Scaled" attention additional divides `wei` by 1/sqrt(head_size). This makes it so when input Q,K are unit variance, wei will be unit variance too and Softmax will stay diffuse and not saturate too much. Illustration below

In [15]:
# Create Head class 
class Head(nn.Module):

    def __init__(self, head_size):
        super().__init__()
        self.key = nn.Linear(n_embd, head_size, bias=False)
        self.query = nn.Linear(n_embd, head_size, bias=False)
        self.value = nn.Linear(n_embd, head_size, bias=False)
        # during training head, query and value are changes

        # tril is not a parameter for our model so we create a buffer
        self.register_buffer('tril', torch.tril(torch.ones(block_size, block_size)))

        # add droupout
        self.dropout = nn.Dropout(dropout)

    def forward(self, x):
        B, T, C = x.shape
        k = self.key(x)
        q = self.query(x)
        # calculate attention score/wei
        wei = q @ k.transpose(-2,-1) * C**(-0.5)
        # masking the wei 
        wei = wei.masked_fill(self.tril[:T, :T]==0, float('-inf'))
        # apply softmax
        wei = F.softmax(wei, dim=-1)
        # dropout layers
        wei = self.dropout(wei)
        # perform the weighted aggregation of value
        v = self.value(x)
        out = wei @ v
        return out

In [16]:
class MultiHeadAttention(nn.Module):
    '''multiple heads of self attention in parallel'''

    def __init__(self, num_heads, head_size):
        super().__init__()
        self.heads = nn.ModuleList([Head(head_size) for _ in range(num_heads)])
        # introduce the residual connection
        # project the output of heads
        self.proj = nn.Linear(n_embd, n_embd)
        # apply dropout
        self.dropout = nn.Dropout(dropout)

    def forward(self, x):
        # concatenate the result of each head
        out = torch.cat([h(x) for h in self.heads], dim=-1)
        out = self.dropout(self.proj(out))
        return out

In [17]:
class FeedForwardBlock(nn.Module):
    '''a simple feed-forward block'''
    def __init__(self, n_embd):
        super().__init__()
        self.net = nn.Sequential(
            nn.Linear(n_embd, 4*n_embd), # increased the output channel 
            nn.ReLU(),
            nn.Linear(4*n_embd, n_embd), # back to same number of channels
            # add dropout for stable model
            nn.Dropout(dropout)
        )
    
    def forward(self, x):
        return self.net(x)

In [18]:
class Block(nn.Module):
    '''Transformer Block'''
    
    def __init__(self, n_embd, n_heads):
        super().__init__()
        head_size = n_embd // n_heads
        self.sa = MultiHeadAttention(n_heads, head_size) # Head(n_embd) where, head_size = n_embd
        self.ffwd = FeedForwardBlock(n_embd)
        self.lay_norm_1 = nn.LayerNorm(n_embd)
        self.lay_norm_2 = nn.LayerNorm(n_embd)

    def forward(self, x):
        x = x + self.sa(self.lay_norm_1(x))
        x = x + self.ffwd(self.lay_norm_2(x))
        return x   

In [19]:
class GPTModel(nn.Module):
    def __init__(self): # remove the vocab_size
        super().__init__()

        # creating a readoff table. Size: vocab_size x vocab_size
        self.token_embedding_table = nn.Embedding(vocabulary_size, n_embd)
        self.position_embedding_table = nn.Embedding(block_size, n_embd)
        # introduce head in the constructor
        # replace the sa_head with multiple attention 
        # self.sa_heads = MultiHeadAttention(4, n_embd//4) # Head(n_embd) where, head_size = n_embd
        # since, we are concatenating the results individual SA block should have n_emdb // 4 channels so that on concatenation, we have n_embd = 32]
        # self.ff = FeedForwardBlock(n_embd)
        self.blocks = nn.Sequential(
            Block(n_embd, n_heads=4),
            Block(n_embd, n_heads=4),
            Block(n_embd, n_heads=4),
            nn.LayerNorm(n_embd),
        )
        self.lm_head = nn.Linear(n_embd, vocabulary_size)

    def forward(self, index, targets=None):
        B, T = index.shape

        token_emb = self.token_embedding_table(index) # (batch, seq length/time, channels); (B, T, C)
        pos_emb = self.position_embedding_table(torch.arange(0, T, device=device)) # (T, C)
        x = token_emb + pos_emb
        x = self.blocks(x) # self.sa_head(x) --> self.sa_heads(x) --> self.blocks(x)
        # x = self.ff(x)# (per node level) which helps the token to think about what they found 
        logits = self.lm_head(x) # (batch, time, vocab_size)

        if targets is None:
            loss = None
        else:
            # read cross_entropy documentation to get the idea about the inputs
            B, T, C = logits.shape
            logits = logits.view(B*T, C) # stretches out the matrix along single dimension
            targets = targets.view(B*T)
            loss = F.cross_entropy(logits, targets) # log-likelihood = -ln(probability)

        return logits, loss
    
    def generate(self, idx, max_new_tokens):
        for _ in range(max_new_tokens):
            # crop idx to the last block_size tokens
            idx_cond = idx[:, -block_size:] # modified this since idx going in my can not exceed th block size
            # get predictions
            logits, loss = self.forward(idx_cond) # or self(index)
            # focus only on last time step
            logits = logits[:, -1, :] # (B, C)
            # apply softmax to get probabilities
            probabs = F.softmax(logits, dim=-1)
            # getting the probability
            index_next = torch.multinomial(probabs, num_samples=1) # (B, 1)
            # append sampled index to the running sequence
            idx = torch.cat((idx, index_next), dim=1) # (B, T+1)
        return idx

How does `view` work?

```python 
a:
tensor([[[5, 0, 9],
         [5, 9, 8],
         [5, 3, 1]],

        [[1, 4, 4],
         [3, 0, 8],
         [3, 8, 9]]])
a_after_view:
tensor([[5, 0, 9],
        [5, 9, 8],
        [5, 3, 1],
        [1, 4, 4],
        [3, 0, 8],
        [3, 8, 9]])
```

#### Logits Explained

1. After processing input data through the layers of a neural network, the final layer generates a set of raw scores or values for each class in a classification task. These raw scores are often referred to as logits.

2. **Purpose:** Logits provide a measure of confidence or certainty that the model assigns to each class. Higher logits indicate greater confidence in a particular class, while lower logits indicate less confidence.

### Step 7: Training the model

In [20]:
# Training
model = GPTModel() # vocabulary_size=88 (The_Great_Gatsby)
m = model.to(device)

# start by defining an optimizer
optimizer = torch.optim.AdamW(model.parameters(), lr=learning_rate)

loss_progress = []

for i in tqdm(range(max_iters), desc="loss:"):

    if i % eval_iters == 0:
        losses = estimate_loss()
        print(f"step: {i}, train loss: {losses['train']:.3f}, val loss: {losses['val']:.3f}")

    # sample a batch of data 
    xb, yb = get_batch('train')

    # evaluate loss
    logits, loss = model.forward(xb, yb)
    optimizer.zero_grad(set_to_none=True)
    loss.backward()
    optimizer.step()
    # loss_progress.append(loss.item())

loss::   0%|          | 0/10000 [00:00<?, ?it/s]

loss::   0%|          | 1/10000 [00:02<7:05:22,  2.55s/it]

step: 0, train loss: 4.673, val loss: 4.674


loss::   2%|▏         | 205/10000 [00:07<15:55, 10.26it/s]

step: 200, train loss: 3.206, val loss: 3.194


...

loss::  96%|█████████▌| 9607/10000 [04:16<00:28, 13.62it/s]

step: 9600, train loss: 2.119, val loss: 2.134


loss::  98%|█████████▊| 9805/10000 [04:21<00:18, 10.31it/s]

step: 9800, train loss: 2.117, val loss: 2.143


loss:: 100%|██████████| 10000/10000 [04:25<00:00, 37.68it/s]


In [22]:
# Lets do some predictions
# Start by feeding some input (get some index)
idx = torch.zeros((1,1), dtype=torch.long, device=device)
generate_values = m.generate(idx=idx, max_new_tokens=500)
print(charwise_decoder(generate_values[0].tolist()))


 dooud stidne. Le,
croke—igh “Thin bech I wer imn aviges the messoped ben beaid an dichimest waorst ody he hon.

Hiler fo he ears, lolly hood the jughitam frooved an’st up tht wione. I hooulf … thith thas suppemsse I dave ing loltixpdine of woled thee at cad thee
ant—walen we es hearn tom jurte
isy alathe kin piveroong ilom r weve
uld, fave ! lettus dimyir arlol thet, whe
was aby’m! horainy rit
wat a hid thing
cborick plinferomve. Docnie ‘y.

“Whlode?”

“I hime eackning affen uV then to thely up
